# Setup do Código

Aqui o espaço para setar todos as variáveis para importar planilha

In [1]:
# Colocar Caminho da Pasta, que armazena os Excels de Consulta (usar '/', para o caminho)
PATH = './src'
CLIENT_PATH = f'{PATH}/Clientes'

# Inserir Nome do Excel, com a extensão
portfolioCliente = 'GVB FIM CP IE - CARTEIRA LOCAL.xlsm'
databaseFundos = 'New_query_2023_11_16 (1).xlsx'
historicoBenchmarks = 'arvore_classificacao_insper.xlsx'

thshRisk = 6.31

<br>

---

<br>

# Rebalanceamento de Carteira

## Importando Bibliotecas

Bibliotecas Importantes usadas para o Código
- Regex (Re): Manipulação de palavras
- Math: Usada para cálculos
- Locale: Importar dados de formatação por região de números
- Numpy: Processamento de Matrizes e Arrays
- Pandas: Manipulação de Dataframes, e planilhas de Excel
- Openpyxl: Leitura e Escrita de Excels
- Scikit-Image (skimage): Processamento de Imagens (processamento de planilhas)

In [2]:
import re
import math
import locale
import numpy as np
import pandas as pd
import openpyxl as xl 
from skimage.measure import label, regionprops

# Setups das Bibliotecas
pd.options.mode.chained_assignment = None  # default='warn'
locale.setlocale(locale.LC_ALL, 'pt-BR.utf-8')

'pt-BR.utf-8'

## Leitura dos Excels e Planilhas:

Função para ler e receber os nomes de cada planilha dentro do excel

In [3]:
# Recebe Caminho do Arquivo, e Nome do Arquivo

def getSheetNames(path, file):
    excel_file = f'{path}/{file}'
    wb = xl.load_workbook(excel_file)
    
    if len(wb.sheetnames) == 1:
        return [wb.sheetnames[0]]
    else:
        return wb.sheetnames


In [4]:
# Carteira Portfólio Cliente
nameClient = getSheetNames(CLIENT_PATH, portfolioCliente)
clientCarteira, clientPolitica = nameClient[0], nameClient[1]

# Base de Dados dos Fundos
nameResult = getSheetNames(PATH, databaseFundos)
result = nameResult[0]

# Árvore Classificação Insper
nameClass = getSheetNames(PATH, historicoBenchmarks)
classBenchmarks, classRetornos = nameClass[0], nameClass[1]

Leitura dos Excels da Empresa

In [5]:
# Carteira do Cliente:
carteira = pd.read_excel(f'{CLIENT_PATH}/{portfolioCliente}', sheet_name=clientCarteira, header=None)
politica = pd.read_excel(f'{CLIENT_PATH}/{portfolioCliente}', sheet_name=clientPolitica, header=None)

# Base de Dados Fundos
database = pd.read_excel(f'{PATH}/{databaseFundos}', sheet_name=result, header=None)

# Árvore Classificação Insper
benchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classBenchmarks, header=None)
retornosBenchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classRetornos, header=None)

## Extração de Tabelas e Atribuição de Variáveis Dataframe

#### Extração de Tabelas dentro de cada Excel:

In [6]:
# Função para Ler as Tabelas dentro de cada Planilha
def getTables(df):
    # Transformando Dataframe em 0s e 1s
    larr = label(np.array(df.notnull()).astype("int"))
    
    tables = []
    for s in regionprops(larr):
        # Filtro de "Tabelas" Pequenas
        ux, lx, uy, ly = s.bbox
        diff_u, diff_l = uy - ux, ly - lx
        if diff_u <= 1 or diff_l <= 1:
            continue
        
        # Criação do Sub-Dataframe 
        table = df.iloc[
            s.bbox[0]:s.bbox[2], s.bbox[1]:s.bbox[3]
        ].pipe(
            lambda df_: df_.rename(columns=df_.iloc[0]).drop(df_.index[0])
        )

        # Adicionando Sub-Dataframe a Lista de Tables
        tables.append(table)
        
    return tables


In [7]:
dfsCarteira = getTables(carteira)
dfsPolitica = getTables(politica)
dfsDatabase = getTables(database)
dfsBenchmarks = getTables(benchmarks)
dfsRetornos = getTables(retornosBenchmarks)

#### Criando Variáveis para cada Tabela dos Excels

In [8]:
def attrDataframes(list_dfs):
    def getVarName(df):
        def namestr(obj, namespace):
            return [name for name in namespace if namespace[name] is obj]
    
        return namestr(df, globals())[0]
    
    for i in range(len(list_dfs)):
        dfName = getVarName(list_dfs).replace('dfs', '')
        
        varName = f'df{dfName}{i+1}'
        value = list_dfs[i]
        globals()[varName] = value
        
        print(f'- Dataframe {varName} created.')
    print('')


In [9]:
attrDataframes(dfsCarteira)
attrDataframes(dfsPolitica)
attrDataframes(dfsDatabase)
attrDataframes(dfsBenchmarks)
attrDataframes(dfsRetornos)

- Dataframe dfCarteira1 created.

- Dataframe dfPolitica1 created.
- Dataframe dfPolitica2 created.

- Dataframe dfDatabase1 created.

- Dataframe dfBenchmarks1 created.

- Dataframe dfRetornos1 created.



In [10]:
dfCarteira1

,ValDate,TradingDesk,Carteira,Quantidade,Financeiro,Classe,%,Mov,Financeiro,%,Resgates Programados,Financeiro,%
4,NaN,NaN,Caixa / Conta Corrente / Provisões,NaN,0.23,NaN,0.0,NaN,0.23,0.0,NaN,0.23,0.0
5,2023-10-31 00:00:00,GVB FIM CP IE - Jera,Cash BRL,0.23,0.23,Caixa / Conta Corrente / Provisões,0.0,NaN,0.23,0.0,NaN,0.23,0.0
6,NaN,NaN,Renda Fixa Brasil CDI,NaN,1447966.429846,NaN,0.132517,NaN,1447966.429846,0.132517,NaN,1447966.429846,0.132517
7,2023-10-31 00:00:00,GVB FIM CP IE - Jera,TESOURO SELIC FI RF,151955.572009,611628.331006,Renda Fixa Brasil CDI,0.055976,NaN,611628.331006,0.055976,NaN,611628.331006,0.055976
8,2023-10-31 00:00:00,GVB FIM CP IE - Jera,LFT01092027,60,836338.09884,Renda Fixa Brasil CDI,0.076541,NaN,836338.09884,0.076541,NaN,836338.09884,0.076541
9,2023-10-31 00:00:00,NaN,Dummy,NaN,NaN,Renda Fixa Brasil CDI,0,NaN,0,0,NaN,0,0
10,NaN,NaN,Renda Fixa Brasil Crédito Pós-Fixado,NaN,2374127.259057,NaN,0.217279,NaN,2374127.259057,0.217279,NaN,2374127.259057,0.217279
11,2023-10-31 00:00:00,GVB FIM CP IE - Jera,JWM CREDITO FIM CP,9875.54888,1427965.886418,Renda Fixa Brasil Crédito Pós-Fixado,0.130686,NaN,1427965.886418,0.130686,NaN,1427965.886418,0.130686
12,2023-10-31 00:00:00,GVB FIM CP IE - Jera,AUGME PRO FIM CP,304794.265204,472891.664346,Renda Fixa Brasil Crédito Pós-Fixado,0.043279,NaN,472891.664346,0.043279,NaN,472891.664346,0.043279
13,2023-10-31 00:00:00,GVB FIM CP IE - Jera,CAPITÂNIA JC CP FIM,390902.036549,473269.708294,Renda Fixa Brasil Crédito Pós-Fixado,0.043313,NaN,473269.708294,0.043313,NaN,473269.708294,0.043313


## Dataframes

### - Carteira do Cliente:

In [11]:
valorCarteira = dfCarteira1["Financeiro"].iloc[-1]
perctCarteira = dfCarteira1["%"].iloc[-1]

In [12]:
dfCarteira1 = dfCarteira1.dropna(subset=["Source"])
dfCarteira1 = dfCarteira1.dropna(subset=["Portfólio"])

KeyError: ['Source']

In [ ]:
dfCarteira1.head()

### - Política de Investimento

In [ ]:
dfPolitica1.head()

In [ ]:
dfPolitica2.rename(columns={'%':'Estratégico'}, inplace=True)
dfPolitica2.head()

In [ ]:
dfPolitica3.head()

### - Base de Dados Fundos: (retirar?)

In [ ]:
dfDatabase1.head()

### - Base de Dados e Histórico Benchmark:

In [ ]:
dfBenchmarks1.head()

In [ ]:
dfRetornos1.head()

## Rebalanceamento do Portfólio:

### Dataframes Importantes para Análise

Dataframe para checar o Rebalanceamento

In [ ]:
dfRebalance = dfPolitica1[['Book', 'Min', 'Max', 'Tático', 'Atual']]
dfRebalance.set_index('Book', inplace=True)
dfRebalance

Dataframe de Análise do Portfólio

In [ ]:
result = dfPolitica1[['Book', 'Tático', 'Atual']]
result['Offset'] = result['Atual'] - result['Tático']

result = result[['Book', 'Tático', 'Atual', 'Offset']]
result.set_index('Book', inplace=True)

result

### Tracking Error do Portfólio:

In [ ]:
retTatico = dfRetornos1.copy()
retTatico.set_index('Data', inplace=True)

for index, row in result.iterrows():
    book = index
    currentPerc = row['Tático']

    retTatico[book] = retTatico[book] * currentPerc

retTatico['Retorno Tático'] = retTatico.sum(axis=1)

# retTatico.head()

In [ ]:
retAtual = dfRetornos1.copy()
retAtual.set_index('Data', inplace=True)

for index, row in result.iterrows():
    book = index
    currentPerc = row['Atual']

    retAtual[book] = retAtual[book] * currentPerc

retAtual['Retorno Atual'] = retAtual.sum(axis=1)
    
# retAtual.head()

In [ ]:
trackingError = pd.DataFrame({
    'Retorno Atual': retAtual['Retorno Atual'],
    'Retorno Tático': retTatico['Retorno Tático']
})

trackingError['Excesso'] = trackingError['Retorno Atual'] - trackingError['Retorno Tático']

trackingError.head()

In [ ]:
te = trackingError['Excesso'].std()
te = te * math.sqrt(12)

print(f"Tracking Error do Portfólio: {te:.3%}")

### Tracking Error Individual por Aplicação:

In [ ]:
matrizCov = dfRetornos1.copy()

matrizCov.reset_index(drop=True)
matrizCov.drop('Data', axis=1, inplace=True)
matrizCov = matrizCov.astype(float)

# matrizCov

In [ ]:
mCov = matrizCov.cov()

# mCov

In [ ]:
pWeights = result["Atual"].to_list()

pVariance = np.dot(pWeights, np.dot(mCov, pWeights))
pVolatility = math.sqrt(pVariance)

print(f"Variância do Portfólio: {pVariance:.5e}")
print(f"Volatilidade do Portfólio: {pVolatility:.5%}")

In [ ]:
covPonderada = np.dot(pWeights, mCov)

# covPonderada

In [ ]:
riskContribuition = []

for i in range(len(covPonderada)):
    riskContribuition.append(pWeights[i] * covPonderada[i]/pVariance)

riskContribuition = list(i * 100 for i in riskContribuition)

rankingRisk = pd.DataFrame({
    'Ativos': result.index.to_list(),
    'Contribuição [%]': riskContribuition
})

rankingRisk = rankingRisk.sort_values(by='Contribuição [%]', ascending= False)

highlight_condition = (rankingRisk['Contribuição [%]'] > thshRisk)
def highlight_row(s):
    return ['background-color: yellow' if v else '' for v in s]

# rankingRisk = rankingRisk.style.apply(highlight_row, subset=highlight_condition, axis=1)
rankingRisk = rankingRisk.style.apply(highlight_row, subset=pd.IndexSlice[highlight_condition, :])

rankingRisk

### Otimização:

## Resultados

In [ ]:

pValue = locale.format_string("%.3f", valorCarteira, grouping=True, monetary=True)
print(f"[Portfolio Value]\n- {pValue}\n")

acoes = []
values = []
for index, row in result.iterrows():
    if row['Offset'] > 0:
        acao = 'Buy' 
    elif row['Offset'] <  0:
        acao = 'Sell'
    else:
        acao = 'Skip'

    price = abs(valorCarteira * row['Offset'])
    price = locale.format_string("%.3f", price, grouping=True, monetary=True)
    print(
f'''[{index}]
- {acao} {price} of Porfolio Value ({abs(row['Offset']):.3%})
'''
    )
    
    acoes.append(acao)
    values.append(price)

result['Order'] = acoes
result['Values'] = values

In [ ]:
result

# Teste

In [ ]:
# importing the openpyxl module as xl 

# excel file used here gfg.xlsx 
excel_file = "./src/Clientes/Portfolio Global - KING.xlsx"

# load the workbook 
wb = xl.load_workbook(excel_file) 

# print the list that stores the sheetnames 
print(wb.sheetnames) 

# we can also get the name of the sheet which is active by using the active property 
# print("Active sheet: ", wb.active) 

# By default the active sheet is the 1st one and can be changed 
wb._active_sheet_index = 4
sheet = wb.active 
# print("Active sheet: ", wb.active) 


In [ ]:
# teste = pd.read_excel(f'{path}/Clientes/GVB FIM CP IE - CARTEIRA LOCAL.xlsm', sheet_name='Nivel_II', header=None)
# teste